# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.26it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [13]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [14]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [15]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [16]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [17]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [18]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Thought: The GitHub profile of João Moura (incorrectly identified as Noah initially) provides substantial information about his technical projects and contributions. Notable projects include 'crewAI', a framework for AI agents, and various other repositories in programming languages like Python, Elixir, and Ruby. I need to incorporate these specifics into the comprehensive profile, aligning with the details provided in the personal write-up about Noah.

Final Answer: Noah is a seasoned Software Engineering Leader with 18 years of experience, holding an MBA and a robust background in AI and data science. He has demonstrated expertise in managing both remote and in-office teams and has a deep knowledge of multiple programming languages and frameworks. Noah's GitHub profile, under the pseudonym João Moura, highlights his contributions to significant projects such as:

1. **crewAI** (Python): A framework for orchestrating role-playing, autonomous AI agents, focusing on collaborative intell

Thought: 
To tailor Noah's resume for the Full Stack Engineer position, I need to update the initial summary, work experience, skills, and education sections. I will emphasize his full stack capabilities, experience in managing software development, proficiency in multiple programming languages, and his leadership in AI-driven projects. Additionally, I will align his characteristics with the job description, such as demonstrating accountability, grit, and ownership.

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a seasoned Full Stack Engineer with 18 years of experience in the tech industry, proficient in developing both desktop and mobile applications. His expertise spans across front-end and back-end technologies, including HTML/CSS, JavaScript, Java, Python, Angular, React, and Node.js. Noah has a strong background in AI, leading initiatives that integrate AI/ML models into scalable web applications. He excels

Thought: 
Based on the job requirements for the Full Stack Engineer position at AI Fund and Noah's detailed resume, I need to develop a set of interview questions and talking points that will help highlight his strengths and experiences relevant to the position. The questions should cover technical skills, project management, leadership, and his unique contributions to past projects.

Final Answer:
**Interview Questions and Talking Points for Noah Williams**

1. **Technical Expertise and Experience**
   - "Can you walk us through your experience with both front-end and back-end development, particularly with technologies listed in our job requirements like HTML/CSS, JavaScript, Java, and Python?"
   - "You've worked extensively with databases such as MySQL and MongoDB. Could you discuss a challenging database issue you resolved?"
   - "In your role at DataKernel, you integrated AI technologies into your projects. How do you approach the integration of AI/ML models in web applications?"

- Dislplay the generated `tailored_resume.md` file.

In [20]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a seasoned Full Stack Engineer with 18 years of experience in the tech industry, proficient in developing both desktop and mobile applications. His expertise spans across front-end and back-end technologies, including HTML/CSS, JavaScript, Java, Python, Angular, React, and Node.js. Noah has a strong background in AI, leading initiatives that integrate AI/ML models into scalable web applications. He excels in project management, UI/UX design, and has a robust understanding of databases like MySQL and MongoDB.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division by integrating full stack development practices, enhancing product capabilities and market positioning with modern web technologies.
- Played a crucial role in adopting front-end and back-end frameworks, leading to improved software solutions and customer engagement.
- Focused on developing and managing well-functioning databases and applications, aligning with AI Fund’s expectations for technical expertise and project delivery.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed full stack development strategies, fostering an environment that balances innovation with robust software engineering practices.
- Managed cross-functional teams in developing APIs and testing software to ensure responsiveness and efficiency, aligning with the responsibilities of a Full Stack Engineer.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Oversaw the development of IoT solutions integrating full stack technologies, from database management with MongoDB to front-end user interfaces, enhancing product functionality and user experience.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Implemented full stack development projects using Ruby on Rails and React, focusing on appealing visual design and efficient APIs which enhanced project outcomes and user satisfaction.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led software development projects that involved extensive use of database and back-end technologies, directly contributing to a 500% increase in company revenue through enhanced software solutions.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed crucial full stack functionalities, including a central API that improved system performance and user engagement, demonstrating strong technical and problem-solving skills.

### BetCraft: CTO — 2013 - 2015
- Guided full stack development efforts using React and Angular, enhancing platform performance and user interface design, which significantly expanded the company's market reach.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for the Full Stack Engineer position at AI Fund, demonstrating a strong blend of technical expertise, leadership, and a results-driven approach. His extensive experience in full stack development and project management makes him a valuable asset to any technology-driven enterprise.

- Dislplay the generated `interview_materials.md` file.

In [21]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Noah Williams**

1. **Technical Expertise and Experience**
   - "Can you walk us through your experience with both front-end and back-end development, particularly with technologies listed in our job requirements like HTML/CSS, JavaScript, Java, and Python?"
   - "You've worked extensively with databases such as MySQL and MongoDB. Could you discuss a challenging database issue you resolved?"
   - "In your role at DataKernel, you integrated AI technologies into your projects. How do you approach the integration of AI/ML models in web applications?"

2. **Project Management and Leadership**
   - "You have held several leadership roles where you managed cross-functional teams. What is your approach to project management and team leadership in a remote setting?"
   - "Could you give an example of a time when you had to lead a project through a difficult phase? What strategies did you employ to navigate those challenges?"
   - "How do you ensure that your projects align with strategic business goals, especially when working with technical and non-technical stakeholders?"

3. **Innovative Solutions and Problem Solving**
   - "Looking at your project 'crewAI', what was the inspiration behind it, and what were the major challenges you faced during its development?"
   - "You've implemented several security measures in your past projects. Could you explain your approach to creating effective security and data protection settings?"
   - "Can you discuss a time when you had to implement a solution with limited resources? How did you manage to maintain the quality and integrity of the project?"

4. **Cultural Fit and Personal Insights**
   - "Our company values accountability and grit, as outlined in our job description. Can you provide examples from your career where you demonstrated these characteristics?"
   - "AI Fund is committed to diversity and inclusion. How have you addressed these important aspects in your previous roles?"
   - "What drives your interest in AI and software architecture, and how do you stay updated with the latest developments in these areas?"

5. **Future Vision and Goals**
   - "Given your extensive experience and leadership skills, where do you see yourself contributing most significantly at AI Fund?"
   - "What are your professional development goals for the next five years, and how does the Full Stack Engineer position at AI Fund align with these goals?"

These questions and talking points are designed to delve into Noah's technical abilities, leadership skills, and alignment with the company's values and goals, ensuring a comprehensive assessment during the interview process.